In [1]:
!pip install -q langchain llama-cpp-python faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf

In [3]:
!pip install -U langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import TextLoader #to load document
from langchain.text_splitter import CharacterTextSplitter # to split into chunks
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
import os

In [7]:
loader=TextLoader("/content/project_text_file.txt")
documents=loader.load()

In [14]:
splitter=CharacterTextSplitter(chunk_size=200,chunk_overlap=0)
docs=splitter.split_documents(documents)

In [15]:
embeddings=HuggingFaceEmbeddings()
vectorestore=FAISS.from_documents(docs,embeddings)

/tmp/ipython-input-15-3212167006.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()


In [21]:
llm=LlamaCpp(
    model_path="/content/tinyllama.gguf",
    temperature=0.75, #creativity of llm
    verbose=False,
    max_token=512,   #models can only have maximum of 512 tokens
    # top_p=0.95,   #to get the sentence meaningfully use top_p
    #n_ctx=2048,     #a model training od maximum tokens
)

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! max_token is not default parameter.
                max_token was transferred to model_kwargs.
                Please confirm that max_token is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [19]:
#Create QA chain
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorestore.as_retriever())

In [20]:
#ask a qestion
question="What role does softmax plays here?"
result=qa_chain.run(question)
print(result)

llama_perf_context_print:        load time =   30738.67 ms
llama_perf_context_print: prompt eval time =   30738.23 ms /   445 tokens (   69.07 ms per token,    14.48 tokens per second)
llama_perf_context_print:        eval time =    7588.80 ms /    66 runs   (  114.98 ms per token,     8.70 tokens per second)
llama_perf_context_print:       total time =   38413.05 ms /   511 tokens


 The softmax function in the model is used to convert the predicted class labels to probabilities. This helps the model choose the most probable class for a given input, ensuring that it predicts accurately and also preventing over-fitting by avoiding class imbalance. In this context, softmax plays a crucial
